In [26]:
import pandas as pd

# Load the dataset to examine its structure and contents
data_path = "C:/Users/amilb/Desktop/Sem 6/IR/Project/Travall/Final Project/Dataset/Combined_datasets/City_Place_RelReview.csv"
data = pd.read_csv(data_path)
data.head()

,City,Place,Relevant Review
0,Delhi,Chandni Chowk,chandni chowk delhi place market food old shop...
1,Delhi,Connaught Place,place connaught delhi shopping shops restauran...
2,Delhi,Delhi Airport Metro Express,airport metro delhi station express new time w...
3,Delhi,Friday Mosque (Jama Masjid),mosque masjid delhi place visit shoes old red ...
4,Delhi,Gurudwara Bangla Sahib,gurudwara place sikh sahib bangla delhi visit ...


In [27]:
import random
import numpy as np
# Function to generate a longer synthetic review
def generate_longer_review(keywords):
    words = keywords.split()
    random.shuffle(words)
    # Ensuring each review is longer by using almost all keywords
    return ' '.join(words[:max(20, len(words))])

# Filter for places with the city label 'Delhi'
delhi_data = data[data['City'] == 'Delhi']

# Generate 500 synthetic reviews specifically for Delhi, using longer reviews
delhi_data['Delhi Synthetic Review'] = delhi_data['Relevant Review'].apply(generate_longer_review)
delhi_data['Label'] = delhi_data['Place']

# Check if there are enough entries for Delhi or if we need to repeat some
delhi_synthetic_data = pd.DataFrame(np.repeat(delhi_data[['Delhi Synthetic Review', 'Label']].values, 1000, axis=0), columns=['Synthetic Review', 'Label'])
delhi_synthetic_data = delhi_synthetic_data.sample(n=500, random_state=42).reset_index(drop=True)
delhi_synthetic_data.head()



C:\Users\amilb\AppData\Local\Temp\ipykernel_37500\4272059951.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delhi_data['Delhi Synthetic Review'] = delhi_data['Relevant Review'].apply(generate_longer_review)
C:\Users\amilb\AppData\Local\Temp\ipykernel_37500\4272059951.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delhi_data['Label'] = delhi_data['Place']


,Synthetic Review,Label
0,area tourist enjoy site building years old hin...,Qutub Minar
1,green visit enjoyed crowded quite years really...,Humayun's Tomb
2,jahan gate chowk audio new area places guide l...,Red Fort (Lal Quila)
3,near items cheap old stalls streets really cro...,Chandni Chowk
4,night water streets want cafe nice area really...,Hauz Khas Village


In [28]:
# save this as evaluation data
delhi_synthetic_data.to_csv("C:/Users/amilb/Desktop/Sem 6/IR/Project/Travall/Final Project/Dataset/Combined_datasets/Delhi_Evaluation Dataset.csv", index=False)

In [29]:
# print the size of the dataset
print("Size of the dataset:", delhi_synthetic_data.shape)

Size of the dataset: (500, 2)


In [30]:
# doing a tf-idf vectorization on initial dataset
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform 
delhi_data = pd.read_csv('C:/Users/amilb/Desktop/Sem 6/IR/Project/Travall/Final Project/Dataset/Dataset2/review_dataset/New Delhi_top_15_places_dataset.csv')

In [31]:
delhi_data.head()

,City,Place,Review,Rating,Name,Date,Raw_Review,Wheelchair Accessibility,Visual Impairments,Mobility Impairments,Hearing Impairments
0,New Delhi,Chandni Chowk,dusty crowded chaotic great walk ride rickshaw...,3,Anonymous,NaN,"Dusty, crowded and chaotic, it's great to have...",0.0,0.0,0.0,1.0
1,New Delhi,Chandni Chowk,times obviously place real india struggle dail...,2,Anonymous,NaN,"I have been there 4 times, obviously I just ca...",0.0,0.0,0.0,1.0
2,New Delhi,Chandni Chowk,came chandi chowk mend late fathers old automa...,5,Anonymous,NaN,I came to Chandi Chowk to find somewhere to me...,0.0,0.0,0.0,1.0
3,New Delhi,Chandni Chowk,day delhi missed got tuc tuc red fort headed m...,2,Anonymous,NaN,We only had 1 day in Delhi but this was not to...,0.0,0.0,0.0,1.0
4,New Delhi,Chandni Chowk,wish experience old delhi charm mad rush peopl...,4,Anonymous,NaN,BE there if you wish to experience the old Del...,0.0,0.0,0.0,1.0


In [32]:
# Aggregate reviews by concatenating them for each place
aggregated_reviews = delhi_data.groupby('Place')['Review'].apply(lambda x: ' '.join(x)).reset_index()

aggregated_reviews.head()

,Place,Review
0,Chandni Chowk,dusty crowded chaotic great walk ride rickshaw...
1,Connaught Place,area good lot eateries available weekends plac...
2,Delhi Airport Metro Express,convenient journey airport city center option ...
3,Friday Mosque (Jama Masjid),review little unfair visited ramadan want insi...
4,Gurudwara Bangla Sahib,delhi way miss visiting place serene magnifice...


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=100)

# Fit and transform the aggregated reviews to create embeddings
tfidf_matrix = tfidf_vectorizer.fit_transform(aggregated_reviews['Review'])

# Convert to array for similarity computation later
tfidf_matrix_array = tfidf_matrix.toarray()

tfidf_matrix_array.shape

(15, 100)

In [34]:
#print tfidf values with their names
tfidf_df = pd.DataFrame(tfidf_matrix_array, columns=tfidf_vectorizer.get_feature_names_out())
#printfirst few 5 rows of the dataframe
tfidf_df.head()

,airport,amazing,architecture,area,beautiful,best,big,building,buildings,built,...,visited,visiting,walk,want,water,way,went,wonderful,world,worth
0,0.000280,0.047495,0.003216,0.111563,0.014100,0.092516,0.017563,0.002226,0.011587,0.007916,...,0.043537,0.028447,0.061100,0.065058,0.003687,0.041805,0.031168,0.014595,0.027210,0.041063
1,0.005907,0.027046,0.033001,0.104463,0.027046,0.084613,0.042927,0.011166,0.036980,0.015384,...,0.026054,0.022332,0.050123,0.042430,0.000792,0.012655,0.021091,0.007196,0.014392,0.024565
2,0.612619,0.013119,0.000234,0.006325,0.004451,0.122053,0.005622,0.000703,0.000000,0.001171,...,0.003045,0.001406,0.011245,0.025301,0.000000,0.133297,0.004451,0.008668,0.031626,0.007965
3,0.000000,0.063036,0.145665,0.114999,0.169516,0.041740,0.060481,0.078369,0.009975,0.152480,...,0.136295,0.072406,0.061333,0.056222,0.007255,0.043444,0.065592,0.019592,0.038333,0.128628
4,0.004114,0.057201,0.021051,0.025987,0.139955,0.062718,0.025407,0.021487,0.002627,0.010308,...,0.078833,0.057201,0.022939,0.021777,0.034310,0.013792,0.032956,0.022503,0.018002,0.030924


In [35]:
from sklearn.metrics.pairwise import cosine_similarity
# Sample user input
user_input = ["mosque historical architecture beautiful peaceful serene religious spiritual"]

# Convert the user input into TF-IDF embedding
user_input_tfidf = tfidf_vectorizer.transform(user_input).toarray()

# Compute the cosine similarity between user input and places
user_similarity_scores = cosine_similarity(user_input_tfidf, tfidf_matrix_array)

# Get the top 5 recommended places based on similarity scores
top_indices = user_similarity_scores.argsort()[0][-10:][::-1]
recommended_places_based_on_input_tfidf = aggregated_reviews.iloc[top_indices]

In [36]:
# getting the most recommended place for all rows in delhi synthetic data and calculating the accuracy
accuracy = 0
for index, row in delhi_synthetic_data.iterrows():
    user_input = [row['Synthetic Review']]
    user_input_tfidf = tfidf_vectorizer.transform(user_input).toarray()
    user_similarity_scores = cosine_similarity(user_input_tfidf, tfidf_matrix_array)
    top_indices = user_similarity_scores.argsort()[0][-1:][::-1]
    recommended_place = aggregated_reviews.iloc[top_indices]['Place'].values[0]
    if recommended_place == row['Label']:
        accuracy += 1

accuracy = accuracy / delhi_synthetic_data.shape[0] * 100
print("Accuracy of the model is:", accuracy, "%")

# finding f1 score, and all the other evaluation metrics
from sklearn.metrics import classification_report

# Get the top recommended place for each synthetic review
predicted_labels = []
for index, row in delhi_synthetic_data.iterrows():
    user_input = [row['Synthetic Review']]
    user_input_tfidf = tfidf_vectorizer.transform(user_input).toarray()
    user_similarity_scores = cosine_similarity(user_input_tfidf, tfidf_matrix_array)
    top_indices = user_similarity_scores.argsort()[0][-1:][::-1]
    recommended_place = aggregated_reviews.iloc[top_indices]['Place'].values[0]
    predicted_labels.append(recommended_place)

# Get the actual labels
actual_labels = delhi_synthetic_data['Label'].values

# Print the classification report
print(classification_report(actual_labels, predicted_labels))


Accuracy of the model is: 66.8 %
                             precision    recall  f1-score   support

              Chandni Chowk       0.43      1.00      0.60        29
            Connaught Place       0.00      0.00      0.00        38
Delhi Airport Metro Express       1.00      1.00      1.00        40
Friday Mosque (Jama Masjid)       0.28      1.00      0.44        37
     Gurudwara Bangla Sahib       0.00      0.00      0.00        35
          Hauz Khas Village       1.00      1.00      1.00        32
             Humayun's Tomb       1.00      1.00      1.00        31
        ISKCON Temple Delhi       0.00      0.00      0.00        32
                 India Gate       1.00      1.00      1.00        30
               Lodhi Garden       1.00      1.00      1.00        31
               Lotus Temple       0.00      0.00      0.00        24
                Qutub Minar       1.00      1.00      1.00        37
         Rashtrapati Bhavan       1.00      1.00      1.00        31


c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# using bert embeddings for the same
from sentence_transformers import SentenceTransformer

# Load the BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Encode the aggregated reviews
aggregated_reviews['Review Embedding'] = aggregated_reviews['Review'].apply(lambda x: model.encode(x))

# Convert the embeddings to a list
aggregated_embeddings = np.array(aggregated_reviews['Review Embedding'].to_list())

# Sample user input
user_input = ["mosque historical architecture beautiful peaceful serene religious spiritual"]

# Encode the user input
user_input_embedding = model.encode(user_input)

# Compute the cosine similarity between user input and places
user_similarity_scores = cosine_similarity(user_input_embedding, aggregated_embeddings)

# Get the top 5 recommended places based on similarity scores
top_indices = user_similarity_scores.argsort()[0][-10:][::-1]
recommended_places_based_on_input_bert = aggregated_reviews.iloc[top_indices]



In [38]:
# getting the most recommended place for all rows in delhi synthetic data and calculating the accuracy
accuracy = 0

for index, row in delhi_synthetic_data.iterrows():
    user_input = [row['Synthetic Review']]
    user_input_embedding = model.encode(user_input)
    user_similarity_scores = cosine_similarity(user_input_embedding, aggregated_embeddings)
    top_indices = user_similarity_scores.argsort()[0][-1:][::-1]
    recommended_place = aggregated_reviews.iloc[top_indices]['Place'].values[0]
    if recommended_place == row['Label']:
        accuracy += 1

accuracy = accuracy / delhi_synthetic_data.shape[0] * 100
print("Accuracy of the model is:", accuracy, "%")

Accuracy of the model is: 77.0 %


In [39]:
# finding f1 score, and all the other evaluation metrics
from sklearn.metrics import classification_report

# Get the top recommended place for each synthetic review
predicted_labels = []
for index, row in delhi_synthetic_data.iterrows():
    user_input = [row['Synthetic Review']]
    user_input_embedding = model.encode(user_input)
    user_similarity_scores = cosine_similarity(user_input_embedding, aggregated_embeddings)
    top_indices = user_similarity_scores.argsort()[0][-1:][::-1]
    recommended_place = aggregated_reviews.iloc[top_indices]['Place'].values[0]
    predicted_labels.append(recommended_place)

# Get the actual labels
actual_labels = delhi_synthetic_data['Label'].values

# Print the classification report
print(classification_report(actual_labels, predicted_labels))

                             precision    recall  f1-score   support

              Chandni Chowk       0.00      0.00      0.00        29
            Connaught Place       0.57      1.00      0.72        38
Delhi Airport Metro Express       1.00      1.00      1.00        40
Friday Mosque (Jama Masjid)       1.00      1.00      1.00        37
     Gurudwara Bangla Sahib       1.00      1.00      1.00        35
          Hauz Khas Village       1.00      1.00      1.00        32
             Humayun's Tomb       0.00      0.00      0.00        31
        ISKCON Temple Delhi       1.00      1.00      1.00        32
                 India Gate       1.00      1.00      1.00        30
               Lodhi Garden       1.00      1.00      1.00        31
               Lotus Temple       0.00      0.00      0.00        24
                Qutub Minar       0.30      1.00      0.46        37
         Rashtrapati Bhavan       0.00      0.00      0.00        31
       Red Fort (Lal Quila)      

c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
# Function to extract the top n words from a document based on TF-IDF
def extract_top_n_words(row, tfidf_vectorizer, n=60):
    # Transform the document to TF-IDF vector
    tfidf_vector = tfidf_vectorizer.transform([row])
    # Get the feature names and tf-idf scores
    feature_names = tfidf_vectorizer.get_feature_names_out()
    sorted_items = sorted(zip(tfidf_vector.toarray().flatten(), feature_names), reverse=True)
    # Get top n words
    top_words = [word for _, word in sorted_items[:n]]
    return ' '.join(top_words)

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=500)  # Considering the top 500 features
vectorizer.fit(delhi_data['Review'])

# Extract top words for each review
aggregated_reviews['Relevant Review'] = aggregated_reviews['Review'].apply(lambda row: extract_top_n_words(row, vectorizer))

In [57]:
aggregated_reviews.head()

,Place,Review,Review Embedding,Relevant Review,Relevant Review Embeddings
0,Chandni Chowk,dusty crowded chaotic great walk ride rickshaw...,"[-0.100976735, 0.9908482, 0.47805464, 0.065212...",chowk chandni market old food delhi place shop...,"[-0.2339497, 0.65138096, 0.4171601, 0.1708652,..."
1,Connaught Place,area good lot eateries available weekends plac...,"[-0.5678539, 0.48516074, 0.449588, 0.20512407,...",place shopping connaught shops cp restaurants ...,"[-0.37223732, 0.28024632, 0.8378291, -0.043822..."
2,Delhi Airport Metro Express,convenient journey airport city center option ...,"[-1.1054823, 0.19444956, 0.35414392, 0.5531221...",airport metro delhi station express new travel...,"[-1.0280166, -0.13376634, 0.40821144, 0.613782..."
3,Friday Mosque (Jama Masjid),review little unfair visited ramadan want insi...,"[-0.032201856, 1.0281426, 0.60659826, 0.100594...",mosque masjid jama shoes old delhi camera larg...,"[-0.2403737, 1.2502785, -0.0054552555, -0.0422..."
4,Gurudwara Bangla Sahib,delhi way miss visiting place serene magnifice...,"[-0.43542767, 1.2588313, 0.09452545, 0.4163538...",gurudwara place sikh sahib visit bangla peace ...,"[-0.0874833, 0.93175316, 0.7865939, -0.0335123..."


In [61]:
# applying sentence transformer on the relevant review and finding the accuracy
# Encode the aggregated reviews
# Load the BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Function to get the embeddings of a sentence
def get_embeddings(sentence):
    return model.encode(sentence)

# Get the embeddings of the relevant reviews
aggregated_reviews['Relevant Review Embeddings'] = aggregated_reviews['Relevant Review'].apply(lambda row: get_embeddings(row))

In [62]:
# now we will calculate the accuracy
# Convert the embeddings to a list
aggregated_embeddings = np.array(aggregated_reviews['Relevant Review Embeddings'].to_list())

# getting the most recommended place for all rows in delhi synthetic data and calculating the accuracy
accuracy = 0
for index, row in delhi_synthetic_data.iterrows():
    user_input = [row['Synthetic Review']]
    user_input_embedding = model.encode(user_input)
    user_similarity_scores = cosine_similarity(user_input_embedding, aggregated_embeddings)
    top_indices = user_similarity_scores.argsort()[0][-1:][::-1]
    recommended_place = aggregated_reviews.iloc[top_indices]['Place'].values[0]
    if recommended_place == row['Label']:
        accuracy += 1

accuracy = accuracy / delhi_synthetic_data.shape[0] * 100
print("Accuracy of the model is:", accuracy, "%")

Accuracy of the model is: 92.60000000000001 %


In [63]:
# print the classification report
# Get the top recommended place for each synthetic review

predicted_labels = []
for index, row in delhi_synthetic_data.iterrows():
    user_input = [row['Synthetic Review']]
    user_input_embedding = model.encode(user_input)
    user_similarity_scores = cosine_similarity(user_input_embedding, aggregated_embeddings)
    top_indices = user_similarity_scores.argsort()[0][-1:][::-1]
    recommended_place = aggregated_reviews.iloc[top_indices]['Place'].values[0]
    predicted_labels.append(recommended_place)

# Get the actual labels
actual_labels = delhi_synthetic_data['Label'].values

# Print the classification report
print(classification_report(actual_labels, predicted_labels))

                             precision    recall  f1-score   support

              Chandni Chowk       1.00      1.00      1.00        29
            Connaught Place       1.00      1.00      1.00        38
Delhi Airport Metro Express       1.00      1.00      1.00        40
Friday Mosque (Jama Masjid)       1.00      1.00      1.00        37
     Gurudwara Bangla Sahib       1.00      1.00      1.00        35
          Hauz Khas Village       1.00      1.00      1.00        32
             Humayun's Tomb       0.46      1.00      0.63        31
        ISKCON Temple Delhi       1.00      1.00      1.00        32
                 India Gate       1.00      1.00      1.00        30
               Lodhi Garden       1.00      1.00      1.00        31
               Lotus Temple       1.00      1.00      1.00        24
                Qutub Minar       0.00      0.00      0.00        37
         Rashtrapati Bhavan       1.00      1.00      1.00        31
       Red Fort (Lal Quila)      

c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amilb\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
